In [1]:
import torch
from model import *
from dataloader import *
from utils.pyart import *
import argparse
import numpy as np
from pathlib import Path

data_path='./data/Multi_2dim_log_spiral/fold9/Multi_2dim_log_spiral_910.txt'
checkpoint= './output/0119_3/checkpoint_50.pth'

In [9]:
# load checkpoint
checkpoint = torch.load(checkpoint)
branchNum = checkpoint['branchNum']
input_dim = checkpoint['input_dim']
branchLs = bnum2ls(branchNum)
n_joint = len(branchLs)

# load model
model = Model(branchLs, input_dim)
model.load_state_dict(checkpoint['state_dict'])
model.eval()

# load data
test_data_loader = ToyDataloader(data_path, n_workers = 1, batch = 1, shuffle=False)

# get revAngle,priAngle
revAngle = np.array([]).reshape(-1,n_joint)
priAngle = np.array([]).reshape(-1,n_joint)
for input,_ in test_data_loader:
    
    rev_q_value, pri_q_value = model.q_layer(input)

    rev_q_value = rev_q_value.detach().cpu().numpy()
    pri_q_value = pri_q_value.detach().cpu().numpy()

    revAngle = np.vstack((revAngle,rev_q_value))
    priAngle = np.vstack((priAngle,pri_q_value))


# get targetPose
targetPose = test_data_loader.dataset.label
targetPose = targetPose.detach().cpu().numpy()

# get outputPose
outputPose = np.array([]).reshape(-1,targetPose.shape[1])
for input,_ in test_data_loader:
    outputPose_temp,_,_ = model(input)
    outputPose_temp = outputPose_temp[:,:,0:3,3]
    outputPose_temp = outputPose_temp.reshape(-1,outputPose_temp.size()[1]*outputPose_temp.size()[2])
    outputPose_temp = outputPose_temp.detach().cpu().numpy()[0]
    outputPose = np.vstack((outputPose,outputPose_temp))


In [45]:
''' check Joint position'''

jointNumber = 5
tick = 300

isrev_ = jointNumber % 2 == 1
Jnumber = int( (jointNumber-3)/2 )

rev_q = revAngle[tick-1,:]
pri_q = priAngle[tick-1,:]

rev_q = torch.tensor(rev_q).unsqueeze(0).to(torch.float)
pri_q = torch.tensor(pri_q).unsqueeze(0).to(torch.float)

TrackingSE3, RevSE3, PriSE3 = model.trans_layer(rev_q,pri_q)

if isrev_:
    T = RevSE3[0,Jnumber].unsqueeze(0)
    p = t2p(T)

else:
    T = PriSE3[0,Jnumber].unsqueeze(0)
    p = t2p(T)

print(p.detach().numpy()[0])

[1.1447837 1.0912329 1.4398816]
